In [5]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [27]:
df = px.data.gapminder().query("year==2007")
df.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.828,31889923,974.580338,AFG,4
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
59,Argentina,Americas,2007,75.320,40301927,12779.379640,ARG,32


In [29]:
fig = px.choropleth(df, locations="iso_alpha",
                    color="lifeExp", # lifeExp is a column of gapminder
                    hover_name="country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [49]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import plotly.graph_objects as go

In [8]:
# read in spreadsheets:
path = "World Happiness Report 2015-2016/"
df2015 = pd.read_csv(path+"2015.csv")
df2016 = pd.read_csv(path+"2016.csv")
dfsuicide = pd.read_csv("who_suicide_statistics.csv")

In [9]:
# add year column to happiness data
df2015['year'] = 2015
df2016['year'] = 2016

In [24]:
# combine happiness data
happy = df2015.append(df2016)

In [25]:
# drop statistical columns (some nans)
happy = happy.drop(['Standard Error','Lower Confidence Interval','Upper Confidence Interval'],axis=1)

In [26]:
happy.head()

,Country,Region,Happiness Rank,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual,year
0,Switzerland,Western Europe,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015
1,Iceland,Western Europe,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,2015
2,Denmark,Western Europe,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,2015
3,Norway,Western Europe,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015
4,Canada,North America,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015


In [12]:
# get 2015-2016 from suicide data
suic_filt = dfsuicide[(dfsuicide['year'] == 2015) | (dfsuicide['year'] == 2016)]

In [13]:
# drop nan values
suic_filt = suic_filt.dropna()

In [14]:
# drop sex and age
suic_filt = suic_filt.drop(['sex','age'], axis=1)

In [15]:
# renaming for agreement across dataframes
suic_filt = suic_filt.rename(columns={'country':'Country'})
suic_filt = suic_filt.replace({'United States of America':'United States'})

In [16]:
# sum population and suicides per country per year
suic_red = suic_filt.groupby(['Country', 'year'])['suicides_no','population'].agg('sum')

<ipython-input-16-309303838e21>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [17]:
# add per capita column
suic_red['per_capita'] = suic_red.suicides_no/suic_red.population

In [20]:
# merge data into one frame
whole_df = pd.merge(suic_red,happy,on=['Country','year'])  
whole_df.head()

,Country,year,suicides_no,population,per_capita,Region,Happiness Rank,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Argentina,2015,3073.0,39699624.0,0.000077,Latin America and Caribbean,30,6.574,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.83600
1,Armenia,2015,74.0,2795335.0,0.000026,Central and Eastern Europe,127,4.350,0.76821,0.77711,0.72990,0.19847,0.03900,0.07855,1.75873
2,Armenia,2016,67.0,2784119.0,0.000024,Central and Eastern Europe,121,4.360,0.86086,0.62477,0.64083,0.14037,0.03616,0.07793,1.97864
3,Australia,2015,3027.0,22240785.0,0.000136,Australia and New Zealand,10,7.284,1.33358,1.30923,0.93156,0.65124,0.35637,0.43562,2.26646
4,Austria,2015,1251.0,8219386.0,0.000152,Western Europe,13,7.200,1.33723,1.29704,0.89042,0.62433,0.18676,0.33088,2.53320


In [21]:
#countries with both happiness and suicide data in 2015 and 2016
whole_df_2 = whole_df.groupby("Country").filter(lambda x: x.Country.size == 2)
whole_df_2.head()

,Country,year,suicides_no,population,per_capita,Region,Happiness Rank,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
1,Armenia,2015,74.0,2795335.0,0.000026,Central and Eastern Europe,127,4.350,0.76821,0.77711,0.72990,0.19847,0.03900,0.07855,1.75873
2,Armenia,2016,67.0,2784119.0,0.000024,Central and Eastern Europe,121,4.360,0.86086,0.62477,0.64083,0.14037,0.03616,0.07793,1.97864
4,Austria,2015,1251.0,8219386.0,0.000152,Western Europe,13,7.200,1.33723,1.29704,0.89042,0.62433,0.18676,0.33088,2.53320
5,Austria,2016,1204.0,8319211.0,0.000145,Western Europe,12,7.119,1.45038,1.08383,0.80565,0.54355,0.21348,0.32865,2.69343
10,Croatia,2015,739.0,4001669.0,0.000185,Central and Eastern Europe,62,5.759,1.08254,0.79624,0.78805,0.25883,0.02430,0.05444,2.75414


In [42]:
path2 = "ne_110m_admin_0_countries/"
shapefile = path2+'ne_110m_admin_0_countries.shp'

In [43]:
#Read shapefile using Geopandas
import geopandas as gpd
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]

#Rename columns.
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()

,country,country_code,geometry
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,SAH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [44]:
print(gdf[gdf['country'] == 'Antarctica'])
#Drop row corresponding to 'Antarctica'
gdf = gdf.drop(gdf.index[159])

gdf = gdf.rename(columns={'country':'Country'})

merged = gdf.merge(happy, on = 'Country')

        country country_code  \
159  Antarctica          ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  


In [45]:
merged.head()

,Country,country_code,geometry,Region,Happiness Rank,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual,year
0,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015
1,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,6,7.404,1.44015,1.09610,0.82760,0.57370,0.31329,0.44834,2.70485,2016
2,Kazakhstan,KAZ,"POLYGON ((87.35997 49.21498, 86.59878 48.54918...",Central and Eastern Europe,54,5.855,1.12254,1.12241,0.64368,0.51649,0.08454,0.11827,2.24729,2015
3,Kazakhstan,KAZ,"POLYGON ((87.35997 49.21498, 86.59878 48.54918...",Central and Eastern Europe,54,5.919,1.22943,0.95544,0.57386,0.40520,0.11132,0.15011,2.49325,2016
4,Uzbekistan,UZB,"POLYGON ((55.96819 41.30864, 55.92892 44.99586...",Central and Eastern Europe,44,6.003,0.63244,1.34043,0.59772,0.65821,0.30826,0.22837,2.23741,2015


In [46]:
whole_df_3 = pd.merge(suic_red,merged,on=['Country','year'])  
whole_df_3.head()

,Country,year,suicides_no,population,per_capita,country_code,geometry,Region,Happiness Rank,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Argentina,2015,3073.0,39699624.0,0.000077,ARG,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000...",Latin America and Caribbean,30,6.574,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.83600
1,Armenia,2015,74.0,2795335.0,0.000026,ARM,"POLYGON ((46.50572 38.77061, 46.14362 38.74120...",Central and Eastern Europe,127,4.350,0.76821,0.77711,0.72990,0.19847,0.03900,0.07855,1.75873
2,Armenia,2016,67.0,2784119.0,0.000024,ARM,"POLYGON ((46.50572 38.77061, 46.14362 38.74120...",Central and Eastern Europe,121,4.360,0.86086,0.62477,0.64083,0.14037,0.03616,0.07793,1.97864
3,Australia,2015,3027.0,22240785.0,0.000136,AUS,"MULTIPOLYGON (((147.68926 -40.80826, 148.28907...",Australia and New Zealand,10,7.284,1.33358,1.30923,0.93156,0.65124,0.35637,0.43562,2.26646
4,Austria,2015,1251.0,8219386.0,0.000152,AUT,"POLYGON ((16.97967 48.12350, 16.90375 47.71487...",Western Europe,13,7.200,1.33723,1.29704,0.89042,0.62433,0.18676,0.33088,2.53320


In [47]:
len(whole_df_3)

61

In [48]:
fig = px.choropleth(whole_df_3, locations="country_code",
                    color="population", # lifeExp is a column of gapminder
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [ ]:
for col in whole_df_3.columns:
    whole_df_3[col] = whole_df_3[col].astype(str)

whole_df_3['text'] = whole_df_3['Country'] + '<br>' + \
    'Suicides per Capita ' + whole_df_3['per_capita'] + '<br>' + \
    'Happiness Rank ' + whole_df_3['Hapiness Rank'] + '<br>' + \
    'Happiness Score ' + whole_df_3['Happiness Score'] + '<br>' + \
    'Veggies ' + whole_df_3['total veggies'] + '<br>' + \
    'Wheat ' + df['wheat'] + '<br>' + \ 
    'Corn ' + whole_df_3['corn']

In [50]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
df.head()

,code,state,category,total exports,beef,pork,poultry,dairy,fruits fresh,fruits proc,total fruits,veggies fresh,veggies proc,total veggies,corn,wheat,cotton
0,AL,Alabama,state,1390.63,34.4,10.6,481.0,4.06,8.0,17.1,25.11,5.5,8.9,14.33,34.9,70.0,317.61
1,AK,Alaska,state,13.31,0.2,0.1,0.0,0.19,0.0,0.0,0.00,0.6,1.0,1.56,0.0,0.0,0.00
2,AZ,Arizona,state,1463.17,71.3,17.9,0.0,105.48,19.3,41.0,60.27,147.5,239.4,386.91,7.3,48.7,423.95
3,AR,Arkansas,state,3586.02,53.2,29.4,562.9,3.53,2.2,4.7,6.88,4.4,7.1,11.45,69.5,114.5,665.44
4,CA,California,state,16472.88,228.7,11.1,225.4,929.95,2791.8,5944.6,8736.40,803.2,1303.5,2106.79,34.6,249.3,1064.95
